In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import csv
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Nguyen Dinh Phuc Dai/Build_Model_Segment/segment.csv', encoding='unicode_escape', index_col = 'cv')

skills = pd.read_csv('/content/drive/MyDrive/Nguyen Dinh Phuc Dai/Data/skills_data.csv',  encoding='latin')
SKILLS_DB = list(set(skills['Skills'].values))

In [4]:
data = data[10:20]

In [5]:
data

,Information,Skill,Education,Experience,SU tag,O tag
cv,,,,,,
11,"+849 39 66 88 39, tedle1997@yahoo.com, linkedi...","python numpy pandas., skills, programming la...","2017 2018 udacity deep learning nanodegree,...","projects , projects , stored managed data in s...",lê lam thang,"machine learning, employment, . unsupervised ..."
12,"dinh thi thu hang, contact information, female...","understanding of computer to findalocation at,...","education, oct 2014 now, post and telecommuni...","01654256960, work experience, ptit, senior stu...","objective, take advantage of it skills & exper...","december 19 1996, language english, interests..."
13,"thieu quang toan, 49 street1 ward3 go vap dist...","skills , micro controller8 32 bits avr arm7 ...","september 2002 to april 2007, bachelor of el...","professional experiences , january 2013 to pre...",NaN,"curriculum vitae, toanquangthieu@gmai.com, toa..."
14,"contact, mail hung. huynh0710 @ gmail . com","learnnew things everyda y, good communicatio...","education, university of information technolog...","phone0898019672, dob07101996, language, read ...",fresh in the industry with enthusiasm to impro...,"englishadvanced, hobby, personal statement, ac..."
15,"truong thanh sang, ttsang8197@gmail.com, 64 do...","scada control and communication, c c++ c#, pl...","electrical electronics and, education, bach k...","automatic engineer, +84934052465, experience a...","career objectives, knowledge for the company o...","as an engineer my target is to gain, profitab..."
16,"le minh binh, district8 ho chi minh city, add...","technical skills , programming language pytho...","university of science ho chi minh city, nd, ed...","achievements responsibilities , rd, pre profes...","career objective , seekingadynamic studying an...",3 prize provi ncial exc ellent student maths c...
17,"hannah smalley, hsmalley@princeton.edu 615 6...","technical skills, programming and computers p...","education, princeton university princeton n...","work & volunteer experience, student tour guid...",NaN,"academic honors, mary george freshman research..."
18,"hiep dang, full name, nationality, date of bir...","skills, solid experience in java php program...","education, bachelor of electric electronic eng...","viet nam, migration design, internal framewor...","summary, 3+ years of software development exp...","languages , april 24 1989, vietnamese native ..."
19,"personal information , full name vu dinh trun...",working hard sociable friendly patient and...,"education , ready for graduating of electronic...","work experience , 8 201132012. i am deliverer...",NaN,"curriculum vitae, social skills , high sense o..."


In [6]:
def extract_skills(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"', " ", str(input_text))
    
    word_tokens = nltk.tokenize.word_tokenize(word_tokens)
    
    
    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]

    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]

    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))

    # create a set to keep the results in.
    found_skills = set()

    # search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in SKILLS_DB:
            found_skills.add(token)

    # search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.lower() in SKILLS_DB:
            found_skills.add(ngram)

    return found_skills

In [7]:
#Extract skill from column Skill

df_skill = pd.DataFrame(columns=['skill', 'extract_skill'])
for i, j in zip(data['Skill'], data['Experience']):
  
    skills1 = extract_skills(i)
    skills2 = extract_skills(j)

    z = str(i) + ''.join(str(j))
    join_skills = skills1.union(skills2)
    df_skill = df_skill.append({'skill': z, 'extract_skill': join_skills}, ignore_index=True)

#Extract skill from column Skill

##df_skill = pd.DataFrame(columns=['skill', 'extract_skill'])
#for i, j in zip(data['Skill'], data['Experience']):
#  while True:
#    try:
#      skills1 = extract_skills(i)
#      skills2 = extract_skills(j)
#    except StopIteration:
#      break
#    z = str(i) + ''.join(str(j))
#    join_skills = skills1.union(skills2)
#    df_skill = df_skill.append({'skill': z, 'extract_skill': join_skills}, ignore_index=True)

In [8]:
#Extract degree and major from column Education using regex

edu_pattern = re.compile('college|master|bachelor|bachelors|.*major.*|.*speciality.*')

df_edu = pd.DataFrame(columns=['education', 'extract_edu'])

for i in data['Education']:
    lst_edu = []
    lst = str(i).split(',')

    for j in lst:
        edu = edu_pattern.findall(j)
        if bool(edu) == True:
            lst_edu.extend(edu)
        
    df_edu = df_edu.append({'education': i, 'extract_edu': lst_edu}, ignore_index=True)

In [9]:
# ([2]{1}[0]{1}[0-9]{2})\s+[0-9]{2}\b
# \b[a-z]{3}\s+ 
# ([2]{1}[0]{1}[0-9]{2})
# ([2]{1}[0]{1}[0-9]{2})

year_pattern = re.compile('([2]{1}[0]{1}[0-9]{2})')
exp_pattern = re.compile('([0-9]{2})\s+year')

df_exp = pd.DataFrame(columns=['experience', 'extract_exp'])

for i, j in zip(data['Experience'], data['SU tag']):
    lst_exp = []
    text = str(i) + ' ' + str(j)
    exp = exp_pattern.findall(text)
    years = year_pattern.findall(text)
    
    if bool(exp)==True:
        to_int = [int(i) for i in exp]
        extract_year = exp
        n_exp = max(to_int)
    else:
        to_int = [int(i) for i in years]
        extract_year = years
        if len(to_int) > 0:
            max_exp = max(to_int)
            min_exp = min(to_int)
            n_exp = max_exp - min_exp
        else:
            n_exp = 'None'
            
    lst_exp.append(str(n_exp))
    
    df_exp = df_exp.append({'experience': text, 'extract_exp': lst_exp}, ignore_index=True)

In [10]:
extracted_data_skill = pd.DataFrame({'Skill': df_skill['skill'], 'Extract_Skill': df_skill['extract_skill']})
extracted_data_skill[0:10]                                                     

,Skill,Extract_Skill
0,"python numpy pandas., skills, programming la...","{data structures, computer science, machine le..."
1,"understanding of computer to findalocation at,...","{raspberry pi, electronics, team working, hard..."
2,"skills , micro controller8 32 bits avr arm7 ...","{vhdl, electronics, orcad, firmware, com, hard..."
3,"learnnew things everyda y, good communicatio...","{weka, machine learning, learnnew things, nosq..."
4,"scada control and communication, c c++ c#, pl...","{scada, orcad, self improvement, teamwork, ms ..."
5,"technical skills , programming language pytho...","{word, tensorflow, powerpoint, mathematical pr..."
6,"technical skills, programming and computers p...","{drawing, arabic, html, css, spanish, english,..."
7,"skills, solid experience in java php program...","{php, eclipse, dbms, sql, html, css, microsoft..."
8,working hard sociable friendly patient and...,"{team work, sourcing}"
9,technical skills ...,"{typescripts, data structures, machine learnin..."


In [11]:
extracted_data_edu = pd.DataFrame({'Education': df_edu['education'], 'Extract_Edu': df_edu['extract_edu']})
extracted_data_edu[0:10]

,Education,Extract_Edu
0,"2017 2018 udacity deep learning nanodegree,...",[college]
1,"education, oct 2014 now, post and telecommuni...",[ major electronic & electrical engineering]
2,"september 2002 to april 2007, bachelor of el...",[ bachelor of electrical and electronics engin...
3,"education, university of information technolog...",[ major in information systems]
4,"electrical electronics and, education, bach k...",[]
5,"university of science ho chi minh city, nd, ed...",[ major mathematical computing.]
6,"education, princeton university princeton n...",[]
7,"education, bachelor of electric electronic eng...",[bachelor]
8,"education , ready for graduating of electronic...",[]
9,education ...,[bachelor]


In [12]:
extracted_data_edu = pd.DataFrame({'Experience': df_exp['experience'], 'Extract_Exp': df_exp['extract_exp']})
extracted_data_edu[0:10]

,Experience,Extract_Exp
0,"projects , projects , stored managed data in s...",[2]
1,"01654256960, work experience, ptit, senior stu...",[2]
2,"professional experiences , january 2013 to pre...",[6]
3,"phone0898019672, dob07101996, language, read ...",[2]
4,"automatic engineer, +84934052465, experience a...",[None]
5,"achievements responsibilities , rd, pre profes...",[1]
6,"work & volunteer experience, student tour guid...",[3]
7,"viet nam, migration design, internal framewor...",[3]
8,"work experience , 8 201132012. i am deliverer...",[4]
9,model. i have experienced programming android ...,[1]


In [13]:
extracted_data = pd.DataFrame({'Skill': df_skill['skill'],
                              'Extract_Skill': df_skill['extract_skill'],
                               'Education': df_edu['education'],
                               'Extract_Edu': df_edu['extract_edu'],
                              'Experience': df_exp['experience'],
                              'Extract_Exp': df_exp['extract_exp'],
                              })
extracted_data[1:50]

,Skill,Extract_Skill,Education,Extract_Edu,Experience,Extract_Exp
1,"understanding of computer to findalocation at,...","{raspberry pi, electronics, team working, hard...","education, oct 2014 now, post and telecommuni...",[ major electronic & electrical engineering],"01654256960, work experience, ptit, senior stu...",[2]
2,"skills , micro controller8 32 bits avr arm7 ...","{vhdl, electronics, orcad, firmware, com, hard...","september 2002 to april 2007, bachelor of el...",[ bachelor of electrical and electronics engin...,"professional experiences , january 2013 to pre...",[6]
3,"learnnew things everyda y, good communicatio...","{weka, machine learning, learnnew things, nosq...","education, university of information technolog...",[ major in information systems],"phone0898019672, dob07101996, language, read ...",[2]
4,"scada control and communication, c c++ c#, pl...","{scada, orcad, self improvement, teamwork, ms ...","electrical electronics and, education, bach k...",[],"automatic engineer, +84934052465, experience a...",[None]
5,"technical skills , programming language pytho...","{word, tensorflow, powerpoint, mathematical pr...","university of science ho chi minh city, nd, ed...",[ major mathematical computing.],"achievements responsibilities , rd, pre profes...",[1]
6,"technical skills, programming and computers p...","{drawing, arabic, html, css, spanish, english,...","education, princeton university princeton n...",[],"work & volunteer experience, student tour guid...",[3]
7,"skills, solid experience in java php program...","{php, eclipse, dbms, sql, html, css, microsoft...","education, bachelor of electric electronic eng...",[bachelor],"viet nam, migration design, internal framewor...",[3]
8,working hard sociable friendly patient and...,"{team work, sourcing}","education , ready for graduating of electronic...",[],"work experience , 8 201132012. i am deliverer...",[4]
9,technical skills ...,"{typescripts, data structures, machine learnin...",education ...,[bachelor],model. i have experienced programming android ...,[1]
